# Forecasting Script - Time Series Analysis in R

The script below allows to forecast Time Series data based on the Forecast package in R, using a mix of both **Exponential Smoothing** and **ARIMA**.

Steps outlined below - 
* Step 1: Defining the function - the auto.ts function provides a way of placing both models (auto.ets and auto.arima) in competition against one another, based on the aic of the model. The test is conducted on x periods determined by the "holdout" parameter. Typically, if holdout = 3, the models will train on all periods except the three most recent, and use those periods to test the accuracy of the model. The function will keep the model with the highest AIC.
* Step 2: Data cleaning - in order for the data to be processed, it needs to be structured in a standard way. More about this in the "README" file below. Provided the data is structured well enough, the script will proceed to transform the csv into a time series component, needed to conduct the analysis.
* Step 3: The script will call the auto.ts function to forecast future elements, and display the resulting chart using ggplot. 


# README

To set-up this forecast script:
* 1) Download your data to the csv format. Make sure your dataset comprises of 3 columns: "YEAR", "MONTH", "OBSERVATIONS". The OBSERVATIONS column can contain anything (e.g average stock price, volume of product sales, etc)
* 2) Run the script using your favorite R installation (mine is Rstudio).
* 3) The script will prompt you to enter a holdout period - this is the number of months you would like to leave out of the training set and serve as a test set to evaluate the models. If Holdout = 0, the accuracy (AIC) will be computed based on the training data to estimate which model provides the best fit.
* 4) The script will prompt you, asking if you would like to export the file - if you chose yes, it will then ask you to chose a directory to store the forecast. Your forecast will be uploaded to this directory with the name "forecast.csv"

In [ ]:
#SCRIPT START - AUTO FORECASTS

#Option + Functions ------------------------------------------------------------------------------------------------------

#disable warning messages (a lot will be called during the script execution because of the NAs in the plot data)
options(warn=-1)

#Load all required packages and libraries
library(RMySQL)
library(forecast)
library(ggplot2)
library(scales)
library(reshape)
library(xlsx)

#Initiate auto-forecast function. This function creates a forecast based on the best fit between the data and the forecast and whooses dynamically between arima or auto.ets
auto.ts <- function(x,ic="aic",holdout=0) {
  S<-start(x)[1]+(start(x)[2]-1)/frequency(x) #Convert YM vector to decimal year
  E<-end(x)[1]+(end(x)[2]-1)/frequency(x)
  holdout<-holdout/frequency(x) #Convert holdout in months to decimal year
  fitperiod<-window(x,S,E-holdout) #Determine fit window
  
  if (holdout==0) {
    testperiod<-fitperiod
  }
  else {
    testperiod<-window(x,E-holdout+1/frequency(x),E) #Determine test window
  }
  
  XP=ets(fitperiod, ic=ic)
  AR=auto.arima(fitperiod, ic=ic)
  
  if (holdout==0) {
    AR_acc<-accuracy(AR)
    XP_acc<-accuracy(XP)
  }
  else {
    AR_acc<-accuracy(forecast(AR,holdout*frequency(x)),testperiod)
    XP_acc<-accuracy(forecast(XP,holdout*frequency(x)),testperiod)
  }
  
  if (AR_acc[3]<XP_acc[3]) { #Use MAE
    model<-AR
    modeltype = "ARIMA"
    mae <- AR_acc[3]
    mape <- AR_acc[4]
  }
  else {
    model<-XP
    modeltype = "Exponential Smoothing"
    mae<-XP_acc[3]
    mape<-XP_acc[4]
  }
  list(model,modeltype,mae,mape,holdout*frequency(x))
}

#clean data for preparation in plotting
funggcast<-function(dn,fcast){ 
  require(zoo) #needed for the 'as.yearmon()' function
  
  en<-max(time(fcast$mean)) #extract the max date used in the forecast
  
  #Extract Source and Training Data
  ds<-as.data.frame(window(dn,end=en,extend=TRUE))
  names(ds)<-'observed'
  ds$date<-as.Date(time(window(dn,end=en,extend=TRUE)))
  
  #Extract the Fitted Values (need to figure out how to grab confidence intervals)
  dfit<-as.data.frame(fcast$fitted)
  names(dfit)[1]<-'fitted'
  dfit$date<-as.Date(time(fcast$fitted))
  
  
  ds<-merge(ds,dfit,all.x=T) #Merge fitted values with source and training data
  
  #Exract the Forecast values and confidence intervals
  dfcastn<-as.data.frame(fcast)
  dfcastn$date<-as.Date(as.yearmon(row.names(dfcastn)))
  names(dfcastn)<-c('forecast','lo80','hi80','lo95','hi95','date')
  
  pd<-merge(ds,dfcastn,all.x=T) #final data.frame for use in ggplot
  return(pd)
  
}

# turn a date into a 'monthnumber' relative to an origin
monnb <- function(d) { lt <- as.POSIXlt(as.Date(d, origin="1900-01-01"));
lt$year*12 + lt$mon } 
# compute a month difference as a difference between two monnb's
mondf <- function(d1, d2) { monnb(d2) - monnb(d1) - 1 }

#Functions above  
#----------------------------------------------------------------------------------------------------------------------------
#Forecast Script below

#get user login
holdout <- readline("Please choose your holdout period -----> ")
holdout <- as.numeric(holdout)
#get data from file
mydata = file.choose()

#import dataset
rs = read.csv(mydata)
data_unt <- rs

#Initiate data_frame and populate - this is to avoid skipping months (going from 4 to 6, we'd rather have 4,5,6, even if 5 is populated with 0 observations)
matrow = mondf("2014-01-01",Sys.Date())+1
data_int <- data.frame(matrix(0,ncol = 3, nrow = matrow))
colnames(data_int) = c("REPORT_YEAR","REPORT_MONTH","OBSERVATIONS")

int = -1
for (z in 1:matrow) {
  if (z%%12 == 0) {
    data_int[z,2] = 12
  }
  else {
    data_int[z,2] = z%%12
  }
  if (z %% 12 == 1) {
    int = int+1
  }
  data_int[z,1] = 2014 + int 
}

int = -1
for (z in 1:matrow) {      
  if (z %% 12 == 1) {
    int = int+1
  }
  year = 2014 + int      
  if (z%%12 == 0) {        
    month = 12        
  } else {    
    month = z%%12    
  }
  if (length(subset(data_unt,data_unt$REPORT_YEAR == year & data_unt$REPORT_MONTH == month,select=OBSERVATIONS)[[1]]) == 0) {
    data_int[z,3] = 0
  } else {
    data_int[z,3] = subset(data_unt,data_unt$REPORT_YEAR == year & data_unt$REPORT_MONTH == month,select=OBSERVATIONS)[[1]] 
  }
}

data <- data_int
mindata <- subset(data, REPORT_YEAR == min(data$REPORT_YEAR))
maxdata <- subset(data, REPORT_YEAR == max(data$REPORT_YEAR))

#Transform data extract into time series
ts = ts(data$OBSERVATIONS,start=c(min(data$REPORT_YEAR),min(mindata$REPORT_MONTH)),
        end=c(max(data$REPORT_YEAR),max(maxdata$REPORT_MONTH)),frequency=12)

#Run forecast
autots = auto.ts(ts,holdout=holdout)
fcst=forecast(autots[[1]],h=18)

#transform data 
pld = funggcast(ts,fcst)
pld$holdout = autots[[5]]
pld$modeltype = autots[[2]]
pld$mae=autots[[3]]
pld$mape = autots[[4]]

p1a<-ggplot(data=pld,aes(x=date,y=observed)) 
p1a<-p1a+geom_line(col="red")
p1a<-p1a+geom_line(aes(y=fitted),col="blue")
p1a<-p1a+geom_line(aes(y=forecast))+geom_ribbon(aes(ymin=lo95,ymax=hi95),alpha=.1)
p1a<-p1a+geom_line(aes(y=forecast))+geom_ribbon(aes(ymin=lo80,ymax=hi80),alpha=.25)
p1a<-p1a+scale_y_continuous(name="Units of Y")
p1a<-p1a+labs(x="Date")
p1a<-p1a+ggtitle(paste(autots[[2]],'Fit to Simulated Data\n (black=forecast, blue=fitted, red=data, dark shadow = 80% conf. interval, light shadow = 95% conf. interval)') )

#Prepare data for export in DB
DATA_EXPORT = melt(pld,id=c("date","lo80","hi80","lo95","hi95","holdout","mae","mape","modeltype"))
DATA_EXPORT = DATA_EXPORT[c(1,9,7,8,6,10,11,2,3,4,5)]

#Display Data
print(p1a)

#Close all connections open
lapply( dbListConnections( dbDriver( drv = "MySQL")), dbDisconnect)

export <- readline("Would you like to write download results as csv (Y/N) ? -----> ")
if (export == "Y") { 
  write.xlsx(DATA_EXPORT, paste(choose.dir(),"/forecast.xlsx",sep=''))
}